In [2]:
!pip install pinecone[grpc] langchain-pinecone langchain_community langchain_openai langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.9 MB/s eta 0:00:00


In [3]:
!pip install -qU langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.1 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [7]:
!pip install gradio

In [37]:
!pip install pypdf

## helper.py

In [35]:
from langchain.document_loaders import TextLoader, DirectoryLoader, UnstructuredFileLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings


# Extract Data from Text, DOCX, and PDF Files
def load_files(data, file_type):
    if file_type == "pdf":
        loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    elif file_type == "txt":
        loader = DirectoryLoader(data, glob="*.txt", loader_cls=TextLoader)
    elif file_type == "docx":
        loader = DirectoryLoader(data, glob="*.docx", loader_cls=UnstructuredFileLoader)
    else:
        raise ValueError("Unsupported file type. Use 'pdf', 'txt', or 'docx'.")

    documents = loader.load()
    return documents


# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


# Download the Embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  # this model returns 384 dimensions
    return embeddings


# # Example Usage
# if __name__ == "__main__":
#     # Specify the directory containing your files
#     data_directory = "data"

#     # Load files (e.g., PDF, TXT, DOCX)
#     file_type = "pdf"  # Change to "pdf" or "docx" as needed
#     documents = load_files(data_directory, file_type)

#     # Split text into chunks
#     text_chunks = text_split(documents)
#     print(f"Total Chunks: {len(text_chunks)}")
#     print("\n"+"#"*30)
#     print(text_chunks[0].page_content)
#     print("\n"+"#"*30)
#     print("\n"+"#"*30)
#     print(text_chunks[1].page_content)
#     print("\n"+"#"*30)
#     print("\n"+"#"*30)
#     print(text_chunks[2].page_content)
#     print("\n"+"#"*30)
#     # Download embeddings
#     embeddings = download_hugging_face_embeddings()
#     print("Embeddings model loaded successfully.")


## store_index.py

In [40]:
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents


load_dotenv()

from google.colab import userdata
key = userdata.get('pinecone')

PINECONE_API_KEY = key
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
extracted_data = load_pdf(data='/content')
text_chunks = text_split(extracted_data)
embeddings = download_hugging_face_embeddings()

index_name = "mesbot"
#Initializing the Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index already exists
try:
    pc.describe_index(index_name)
    print(f"Index '{index_name}' already exists.")
except Exception as e:
    if "not found" in str(e):
        print(f"Creating index '{index_name}'...")
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    else:
        raise  # Re-raise unexpected errors


# Now create the PineconeVectorStore
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

Creating index 'mesbot'...


In [41]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain_pinecone import PineconeVectorStore
import os


# Load embeddings and setup Pinecone
# Uncomment and define `download_hugging_face_embeddings` if needed
# embeddings = download_hugging_face_embeddings()

# Placeholder embeddings logic
embeddings = download_hugging_face_embeddings()

index_name = "mesbot"

# Load Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Load GPT-2 Model and Tokenizer
model_name = "openai-community/gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def retrieve_documents(query):
    """
    Retrieve relevant documents using the retriever.
    """
    docs = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in docs])

def gpt2_generate(prompt, max_length=100, temperature=0.7):
    """
    Generate a response using GPT-2.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        max_new_tokens=max_length*1.5,
        temperature=temperature,
        num_return_sequences=1,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def main():
    print("Welcome to the RAG Chat! Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        # Retrieve relevant documents
        retrieved_docs = retrieve_documents(user_input)
        print(f"Retrieved Docs:\n{retrieved_docs}\n")  # Debugging purposes

        # Combine the user input with retrieved documents
        augmented_prompt = f"Context:\n{retrieved_docs}\n\nQuery: {user_input}\nAnswer:"

        # Generate response
        response = gpt2_generate(augmented_prompt, max_length=200)
        print("RAG: ", response)




In [48]:
import gradio as gr
import re

def extract_answers(text):
    # Use a regular expression to find all occurrences of "\nAnswer:" and capture all text after it
    matches = re.findall(r'\nAnswer:(.*)', text, re.DOTALL)
    return matches



def gpt2_generate(prompt, max_length=100, temperature=0.7):
    """
    Generate a response using GPT-2.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        max_new_tokens=max_length*1.5,
        temperature=temperature,
        num_return_sequences=1,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)





# Function to generate responses and maintain chat history
def chat(prompt):

    # Retrieve relevant documents
    retrieved_docs = retrieve_documents(prompt)
    print(f"Retrieved Docs:\n{retrieved_docs}\n")  # Debugging purposes

    # Combine the user input with retrieved documents
    augmented_prompt = f"Context:\n{retrieved_docs}\n\nQuery: {prompt}\nAnswer:"

    # Generate response
    response = gpt2_generate(augmented_prompt, max_length=200)
    print("RAG: ", response)
    # Append the model's response to the chat history

    # Return the updated chat history
    return response.split("Answer:", 1)[1]

# Create the Gradio interface with updated syntax
iface = gr.Interface(
    fn=chat,                      # Function to call for generating responses
    inputs=gr.Textbox(label="You:", placeholder="Type your message here..."),  # Input component
    outputs=gr.Textbox(label="Bot:"),       # Output component
    title="Chat Bot Application",
    description="Interact with a chatbot model."
)

# Launch the Gradio app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10e121b4fd91029a19.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
